# Census Data Pipeline -- Geographic Hierarchy

Reorganized data pipeline fetching Census ACS + economic data at every level of the Census geographic hierarchy. Each section is self-contained with its own data fetch, save, and visualization.

**Geographic hierarchy** (coarse to fine):
1. **National** -- FRED macro indicators (CPI, interest rates, unemployment, housing starts)
2. **State** -- ACS 5-year demographics + FRED state unemployment
3. **County** -- ACS 5-year demographics + BLS LAUS county unemployment
4. **Tract** -- ACS 5-year demographics + TIGER/Line spatial join + FHFA HPI + OpenCelliD towers
5. **Block Group** -- ACS 5-year demographics (latest year only, finest ACS geography)
6. **ZCTA (ZIP)** -- ACS 5-year demographics + Gazetteer centroids + FHFA HPI ZIP (separate geography)
7. **Temporal Trends** -- 1yr/3yr/5yr growth rates computed from the tract panel
8. **Hierarchical Imputation** -- Fill Census-suppressed values using geographic hierarchy
9. **Summary** -- Pipeline inventory, data quality report

**Year range**: 2015-2024 (10 ACS 5-year vintages), consistent across all levels.
**Features**: 28 raw Census variables + 8 derived FMV features at each level.
**Caching**: Every section checks for cached output files before making API calls.

---
## 0. Setup, Imports & Shared Functions

In [1]:
import os, json, time, warnings
import requests
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

warnings.filterwarnings("ignore", category=FutureWarning)

_cwd = os.getcwd()
_project_root = _cwd if os.path.isdir(os.path.join(_cwd, "data")) or os.path.isfile(os.path.join(_cwd, ".env")) else os.path.dirname(_cwd)
_env_path = os.path.join(_project_root, ".env")

try:
    from dotenv import load_dotenv
    load_dotenv(_env_path)
    print(f".env loaded from: {_env_path}")
except ImportError:
    print("python-dotenv not installed; using environment variables")

CENSUS_API_KEY = os.environ.get("CENSUS_API_KEY", "")
FRED_API_KEY = os.environ.get("FRED_API_KEY", "")
BLS_API_KEY = os.environ.get("BLS_API_KEY", "")
OPENCELLID_API_KEY = os.environ.get("OPENCELLID_API_KEY", "")

print(f"Census API key:    {'SET (' + CENSUS_API_KEY[:8] + '...)' if CENSUS_API_KEY else 'NOT SET'}")
print(f"FRED API key:      {'SET' if FRED_API_KEY else 'NOT SET'}")
print(f"BLS API key:       {'SET' if BLS_API_KEY else 'NOT SET (optional)'}")
print(f"OpenCelliD API key: {'SET' if OPENCELLID_API_KEY else 'NOT SET'}")

DATA_DIR = os.path.join(_project_root, "data")
PROCESSED_DIR = os.path.join(DATA_DIR, "processed")
RAW_DIR = os.path.join(DATA_DIR, "raw")
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(PROCESSED_DIR, exist_ok=True)
os.makedirs(RAW_DIR, exist_ok=True)

.env loaded from: c:\Users\hp\Desktop\Personal_Projects\pj_feb\.env
Census API key:    SET (e2989642...)
FRED API key:      SET
BLS API key:       NOT SET (optional)
OpenCelliD API key: SET


In [2]:
# ---- Timeline: 2015-2024 (10 ACS 5-year vintages) ----
# Defined ONCE here, used by ALL geographic levels for consistency.
YEARS = list(range(2015, 2025))
DATASET = "acs5"
print(f"Timeline: {YEARS[0]}-{YEARS[-1]} ({len(YEARS)} vintages)")

# State FIPS codes (01-56 + 72=PR, excluding unused 03,07,14,43,52)
SKIP_FIPS = {"03", "07", "14", "43", "52"}
ALL_STATE_FIPS = [f"{i:02d}" for i in range(1, 57) if f"{i:02d}" not in SKIP_FIPS]
ALL_STATE_FIPS.append("72")
print(f"State FIPS: {len(ALL_STATE_FIPS)} states/territories")

CENSUS_SENTINEL_VALUES = [
    -666666666, -333333333, -222222222, -999999999,
    -666666, -333333, -222222, -999999, -6666, -3333, -2222, -9999,
]

RENAME_MAP = {
    "B01003_001E": "population", "B19013_001E": "median_income",
    "B25077_001E": "median_home_value", "B25064_001E": "median_rent",
    "B25058_001E": "median_contract_rent",
    "B25002_001E": "total_housing_units", "B25002_003E": "vacant_housing_units",
    "B25003_002E": "owner_occupied_units", "B25003_003E": "renter_occupied_units",
    "B25035_001E": "median_year_built",
    "B25071_001E": "rent_burden_pct", "B25105_001E": "median_monthly_housing_cost",
    "B01002_001E": "median_age", "B19083_001E": "gini_index",
    "B17001_002E": "poverty_population",
    "B08301_001E": "total_commuters", "B08301_010E": "transit_commuters",
    "B08303_001E": "total_workers_commute",
    "B08303_012E": "commute_60_89_min", "B08303_013E": "commute_90_plus_min",
    "B15003_022E": "bachelors_degree", "B15003_023E": "masters_degree",
    "B15003_025E": "doctorate_degree",
    "B23025_002E": "in_labor_force", "B23025_005E": "unemployed_civilian",
    "C24050_002E": "employed_agriculture_mining",
    "C24050_003E": "employed_construction",
    "C24050_029E": "employed_professional_scientific",
    # ---- FMV enrichment: 15 additional variables ----
    "B19301_001E": "per_capita_income",
    "B25003_001E": "total_occupied_units",
    "B08301_003E": "drove_alone",
    "B08301_021E": "worked_from_home",
    "B25024_002E": "single_family_detached",
    "B25024_009E": "large_multifamily_50plus",
    "B25024_010E": "mobile_homes",
    "B25024_001E": "total_units_in_structure",
    "B02001_002E": "pop_white",
    "B02001_003E": "pop_black",
    "B02001_005E": "pop_asian",
    "B02001_008E": "pop_two_or_more_races",
    "B15003_017E": "high_school_diploma",
    "B15003_001E": "education_total",
    "B17001_001E": "poverty_universe",
    # Vacancy breakdown (B25004) for real estate availability
    "B25004_002E": "vacant_for_rent",
    "B25004_004E": "vacant_for_sale",
    "B25004_006E": "vacant_seasonal",
    "B25004_008E": "vacant_other",
}

RAW_FEATURE_NAMES = list(RENAME_MAP.values())
DERIVED_FEATURE_NAMES = [
    "vacancy_rate", "owner_renter_ratio", "transit_commuter_pct",
    "long_commute_pct", "college_degree_pct", "labor_force_participation",
    "poverty_rate", "professional_employment_pct",
    "single_family_pct", "multifamily_pct", "worked_from_home_pct",
    "drove_alone_pct", "diversity_index",
    "for_sale_rate", "for_rent_rate", "abandoned_rate", "available_inventory_rate",
]
print(f"Features: {len(RAW_FEATURE_NAMES)} raw + {len(DERIVED_FEATURE_NAMES)} derived")

FIPS_TO_ABBR = {
    "01": "AL", "02": "AK", "04": "AZ", "05": "AR", "06": "CA",
    "08": "CO", "09": "CT", "10": "DE", "11": "DC", "12": "FL",
    "13": "GA", "15": "HI", "16": "ID", "17": "IL", "18": "IN",
    "19": "IA", "20": "KS", "21": "KY", "22": "LA", "23": "ME",
    "24": "MD", "25": "MA", "26": "MI", "27": "MN", "28": "MS",
    "29": "MO", "30": "MT", "31": "NE", "32": "NV", "33": "NH",
    "34": "NJ", "35": "NM", "36": "NY", "37": "NC", "38": "ND",
    "39": "OH", "40": "OK", "41": "OR", "42": "PA", "44": "RI",
    "45": "SC", "46": "SD", "47": "TN", "48": "TX", "49": "UT",
    "50": "VT", "51": "VA", "53": "WA", "54": "WV", "55": "WI",
    "56": "WY", "72": "PR",
}

Timeline: 2015-2024 (10 vintages)
State FIPS: 52 states/territories
Features: 28 raw + 8 derived


In [3]:
# ---- Shared functions used by every geographic level ----

def fetch_census_two_batch(base_url, geo_params, api_key, rename_map, geo_keys,
                           include_c24050=True):
    """Fetch Census ACS data in two batches (API limit ~25 vars) and merge."""
    batch_a = (
        "B01003_001E,B19013_001E,B25077_001E,B25064_001E,B25058_001E,"
        "B25002_001E,B25002_003E,B25003_002E,B25003_003E,B25035_001E,"
        "B25071_001E,B25105_001E,B01002_001E,B19083_001E,B17001_002E,"
        "B08301_001E,B08301_010E,B08303_001E,B08303_012E,B08303_013E,NAME"
    )
    if include_c24050:
        batch_b = (
            "B15003_022E,B15003_023E,B15003_025E,"
            "B23025_002E,B23025_005E,"
            "C24050_002E,C24050_003E,C24050_029E,NAME"
        )
    else:
        batch_b = "B15003_022E,B15003_023E,B15003_025E,B23025_002E,B23025_005E,NAME"

    params_a = {"get": batch_a, **geo_params}
    params_b = {"get": batch_b, **geo_params}
    if api_key:
        params_a["key"] = api_key
        params_b["key"] = api_key

    r_a = requests.get(base_url, params=params_a, timeout=60)
    r_a.raise_for_status()
    data_a = r_a.json()
    df_a = pd.DataFrame(data_a[1:], columns=data_a[0])

    time.sleep(0.2)
    r_b = requests.get(base_url, params=params_b, timeout=60)
    r_b.raise_for_status()
    data_b = r_b.json()
    df_b = pd.DataFrame(data_b[1:], columns=data_b[0])
    df_merged = df_a.merge(df_b.drop(columns=["NAME"], errors="ignore"),
                           on=geo_keys, how="left")
    if not include_c24050:
        for c in ("employed_agriculture_mining", "employed_construction",
                  "employed_professional_scientific"):
            if c not in df_merged.columns:
                df_merged[c] = np.nan

    # Batch C: FMV enrichment variables (15 additional)
    batch_c = (
        "B19301_001E,B25003_001E,B08301_003E,B08301_021E,"
        "B25024_002E,B25024_009E,B25024_010E,B25024_001E,"
        "B02001_002E,B02001_003E,B02001_005E,B02001_008E,"
        "B15003_017E,B15003_001E,B17001_001E,NAME"
    )

    # Batch D: Vacancy breakdown (B25004) for real estate availability
    batch_d = (
        "B25004_002E,B25004_004E,B25004_006E,B25004_008E,NAME"
    )
    params_c = {"get": batch_c, **geo_params}
    if api_key:
        params_c["key"] = api_key
    time.sleep(0.2)
    r_c = requests.get(base_url, params=params_c, timeout=60)
    r_c.raise_for_status()
    data_c = r_c.json()
    df_c = pd.DataFrame(data_c[1:], columns=data_c[0])
    df_merged = df_merged.merge(df_c.drop(columns=["NAME"], errors="ignore"),
                               on=geo_keys, how="left")

    params_d = {"get": batch_d, **geo_params}
    if api_key:
        params_d["key"] = api_key
    time.sleep(0.2)
    r_d = requests.get(base_url, params=params_d, timeout=60)
    r_d.raise_for_status()
    data_d = r_d.json()
    df_d = pd.DataFrame(data_d[1:], columns=data_d[0])
    df_merged = df_merged.merge(df_d.drop(columns=["NAME"], errors="ignore"),
                               on=geo_keys, how="left")

    df_merged.rename(columns=rename_map, inplace=True)
    numeric_cols = [c for c in rename_map.values() if c in df_merged.columns]
    for col in numeric_cols:
        df_merged[col] = pd.to_numeric(df_merged[col], errors="coerce")
    if numeric_cols:
        df_merged[numeric_cols] = df_merged[numeric_cols].replace(CENSUS_SENTINEL_VALUES, np.nan)
    return df_merged


def compute_fmv_derived(df):
    """Compute 8 derived FMV features with denominator-safe NaN propagation."""
    out = df.copy()
    pop = out["population"].where(out["population"] > 0, np.nan)
    th = out.get("total_housing_units", pd.Series(np.nan, index=out.index)).where(lambda x: x > 0, np.nan)
    ro = out.get("renter_occupied_units", pd.Series(np.nan, index=out.index)).where(lambda x: x > 0, np.nan)
    tc = out.get("total_commuters", pd.Series(np.nan, index=out.index)).where(lambda x: x > 0, np.nan)
    tw = out.get("total_workers_commute", pd.Series(np.nan, index=out.index)).where(lambda x: x > 0, np.nan)
    lf = out.get("in_labor_force", pd.Series(np.nan, index=out.index)).where(lambda x: x > 0, np.nan)

    out["vacancy_rate"] = out.get("vacant_housing_units", np.nan) / th
    out["owner_renter_ratio"] = out.get("owner_occupied_units", np.nan) / ro
    out["transit_commuter_pct"] = out.get("transit_commuters", np.nan) / tc * 100.0
    lc = (out.get("commute_60_89_min", pd.Series(0.0, index=out.index)).fillna(0)
          + out.get("commute_90_plus_min", pd.Series(0.0, index=out.index)).fillna(0))
    out["long_commute_pct"] = lc / tw * 100.0
    ed = (out.get("bachelors_degree", pd.Series(0.0, index=out.index)).fillna(0)
          + out.get("masters_degree", pd.Series(0.0, index=out.index)).fillna(0)
          + out.get("doctorate_degree", pd.Series(0.0, index=out.index)).fillna(0))
    out["college_degree_pct"] = ed / pop * 100.0
    out["labor_force_participation"] = lf / pop * 100.0
    out["poverty_rate"] = out.get("poverty_population", np.nan) / pop * 100.0
    out["professional_employment_pct"] = out.get("employed_professional_scientific", np.nan) / lf * 100.0

    # FMV enrichment derived features
    tus = out.get("total_units_in_structure", pd.Series(np.nan, index=out.index)).where(lambda x: x > 0, np.nan)
    out["single_family_pct"] = out.get("single_family_detached", np.nan) / tus * 100.0
    out["multifamily_pct"] = out.get("large_multifamily_50plus", np.nan) / tus * 100.0
    out["worked_from_home_pct"] = out.get("worked_from_home", np.nan) / tc * 100.0
    out["drove_alone_pct"] = out.get("drove_alone", np.nan) / tc * 100.0

    # Herfindahl-based diversity index: 1 - sum(share^2)
    pop_sq = pop ** 2
    race_ssq = sum(
        out.get(c, pd.Series(0.0, index=out.index)).fillna(0) ** 2
        for c in ["pop_white", "pop_black", "pop_asian", "pop_two_or_more_races"]
    )
    out["diversity_index"] = (1.0 - race_ssq / pop_sq.where(pop_sq > 0, np.nan)).clip(0, 1)


    # Vacancy breakdown derived features (real estate availability)
    vac_rent = out.get("vacant_for_rent", pd.Series(np.nan, index=out.index)).fillna(np.nan)
    vac_sale = out.get("vacant_for_sale", pd.Series(np.nan, index=out.index)).fillna(np.nan)
    vac_seasonal = out.get("vacant_seasonal", pd.Series(np.nan, index=out.index)).fillna(np.nan)
    vac_other = out.get("vacant_other", pd.Series(np.nan, index=out.index)).fillna(np.nan)
    out["for_sale_rate"] = vac_sale / th * 100.0
    out["for_rent_rate"] = vac_rent / th * 100.0
    out["abandoned_rate"] = vac_other / th * 100.0
    out["available_inventory_rate"] = (vac_rent.fillna(0) + vac_sale.fillna(0) + vac_other.fillna(0)) / th * 100.0

    return out


def fetch_acs_level(geo_params, geo_keys, extra_cols_fn=None, label=""):
    """Generic ACS fetch loop over YEARS with caching-friendly structure."""
    frames, errors = [], []
    for yr in YEARS:
        base_url = f"https://api.census.gov/data/{yr}/acs/{DATASET}"
        try:
            df_yr = fetch_census_two_batch(
                base_url, geo_params, CENSUS_API_KEY, RENAME_MAP, geo_keys,
                include_c24050=(yr >= 2012),
            )
            df_yr["year"] = yr
            if extra_cols_fn:
                df_yr = extra_cols_fn(df_yr)
            df_yr = compute_fmv_derived(df_yr)
            frames.append(df_yr)
            print(f"  {yr}: {len(df_yr):,} {label}  [OK]")
        except Exception as e:
            errors.append(f"{yr}: {e}")
            print(f"  {yr}: ERROR - {e}")
        time.sleep(0.3)
    df_all = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
    return df_all, errors


print("Shared functions loaded: fetch_census_two_batch, compute_fmv_derived, fetch_acs_level")

Shared functions loaded: fetch_census_two_batch, compute_fmv_derived, fetch_acs_level


---
## 1. National Level -- FRED Macro Economic Indicators

Macro factors that apply uniformly to all geographies. Joined by `year` as a scalar.

| Series | Description | Frequency |
|--------|-------------|-----------|
| CPIAUCSL | CPI (All Urban Consumers) | Monthly |
| DGS10 | 10-Year Treasury Rate | Daily |
| UNRATE | National Unemployment Rate | Monthly |
| HOUST | Housing Starts (thousands) | Monthly |

In [4]:
# ---- Section 1: FRED National Macro Series ----
fred_path = os.path.join(PROCESSED_DIR, "fred_national_macro.parquet")

if os.path.isfile(fred_path):
    print(f"Loading cached: {fred_path}")
    df_fred_national = pd.read_parquet(fred_path)
    print(f"  {len(df_fred_national):,} rows, columns: {list(df_fred_national.columns)}")
else:
    from fredapi import Fred
    fred = Fred(api_key=FRED_API_KEY)

    series_map = {
        "CPIAUCSL": "CPI (All Urban Consumers)",
        "DGS10": "10-Year Treasury Rate",
        "UNRATE": "National Unemployment Rate",
        "HOUST": "Housing Starts (thousands)",
        "CUSR0000SAH1": "CPI Shelter (housing inflation)",
        "MORTGAGE30US": "30-Year Fixed Mortgage Rate",
        "PERMIT": "Building Permits (new construction)",
        "CSUSHPINSA": "Case-Shiller Home Price Index (national)",
        "COMREPUSQ159N": "Commercial RE Price Index (YoY %)",
        "CUSR0000SEHC01": "CPI Owners Equivalent Rent",
    }

    fred_frames = []
    print("Fetching FRED national macro series...")
    for sid, label in series_map.items():
        try:
            s = fred.get_series(sid, observation_start=f"{min(YEARS)}-01-01")
            df_s = s.dropna().reset_index()
            df_s.columns = ["date", sid]
            fred_frames.append(df_s)
            latest = s.dropna().iloc[-1]
            latest_date = s.dropna().index[-1].strftime("%Y-%m-%d")
            print(f"  [OK] {label:35s} | Latest: {latest:>10.2f} ({latest_date}) | {len(s)} obs")
        except Exception as e:
            print(f"  [ERR] {label:35s} | {e}")

    if fred_frames:
        df_fred_national = fred_frames[0]
        for df_f in fred_frames[1:]:
            df_fred_national = df_fred_national.merge(df_f, on="date", how="outer")
        df_fred_national = df_fred_national.sort_values("date").reset_index(drop=True)
        df_fred_national.to_parquet(fred_path, index=False)
        print(f"\nSaved: {fred_path} ({len(df_fred_national):,} rows)")
    else:
        df_fred_national = pd.DataFrame()
        print("No FRED data retrieved")

Loading cached: c:\Users\hp\Desktop\Personal_Projects\pj_feb\data\processed\fred_national_macro.parquet
  4,103 rows, columns: ['date', 'CPIAUCSL', 'DGS10', 'UNRATE', 'HOUST']


In [5]:
# ---- Viz: FRED Macro Time Series (2x2 subplots) ----
if len(df_fred_national) > 0:
    series_labels = {
        "CPIAUCSL": "CPI (All Urban Consumers)",
        "DGS10": "10-Year Treasury Rate",
        "UNRATE": "National Unemployment Rate",
        "HOUST": "Housing Starts (thousands)",
    }
    avail = [s for s in series_labels if s in df_fred_national.columns]
    fig_fred = make_subplots(rows=2, cols=2, subplot_titles=[series_labels[s] for s in avail],
                             vertical_spacing=0.12, horizontal_spacing=0.08)
    colors = ["#1f77b4", "#ff7f0e", "#d62728", "#2ca02c"]
    positions = [(1,1), (1,2), (2,1), (2,2)]
    for idx, sid in enumerate(avail):
        vals = df_fred_national[["date", sid]].dropna()
        r, c = positions[idx]
        fig_fred.add_trace(go.Scatter(x=vals["date"], y=vals[sid], name=series_labels[sid],
                                      line=dict(color=colors[idx], width=2)), row=r, col=c)
    fig_fred.update_layout(title="FRED Macro Economic Indicators (2010-present)",
                           height=600, width=1000, showlegend=False, margin=dict(t=80))
    fig_fred.show()
else:
    print("No FRED data to plot")

---
## 2. State Level -- ACS Demographics + FRED State Unemployment

**Source**: Census ACS 5-year (28+8 vars) + FRED state unemployment rates (`{STATE}UR`)
**Fetch**: 1 API call per year (10 calls total)
**Join key**: `state` FIPS, `year`

In [6]:
# ---- Section 2: State-Level ACS (2015-2024) ----
state_path = os.path.join(DATA_DIR, "census_state_demographics_2015_2024.csv")

if os.path.isfile(state_path):
    print(f"Loading cached: {state_path}")
    df_states_all = pd.read_csv(state_path)
else:
    print(f"Fetching state-level: {len(YEARS)} years, 28 raw + 8 derived features")
    df_states_all, state_errors = fetch_acs_level(
        {"for": "state:*"}, ["state"], label="states"
    )
    df_states_all.to_csv(state_path, index=False)
    print(f"Saved: {state_path}")
    if state_errors:
        print(f"Errors: {state_errors}")

latest_year = int(df_states_all["year"].max())
df_states = df_states_all[df_states_all["year"] == latest_year].copy()

print(f"\n--- State-Level: {len(df_states_all):,} rows x {len(df_states_all.columns)} cols ---")
print(f"Years: {int(df_states_all['year'].min())}-{latest_year} | States/yr: {len(df_states)}")
df_states_all.head(5)

Loading cached: c:\Users\hp\Desktop\Personal_Projects\pj_feb\data\census_state_demographics_2015_2024.csv

--- State-Level: 520 rows x 39 cols ---
Years: 2015-2024 | States/yr: 52


,population,median_income,median_home_value,median_rent,median_contract_rent,total_housing_units,vacant_housing_units,owner_occupied_units,renter_occupied_units,median_year_built,...,employed_professional_scientific,year,vacancy_rate,owner_renter_ratio,transit_commuter_pct,long_commute_pct,college_degree_pct,labor_force_participation,poverty_rate,professional_employment_pct
0,2988081,39665,103100,717,528,1289704,193111,749982,346611,1982,...,214247,2015,0.149733,2.163757,0.399981,6.619805,12.581988,45.279663,21.804797,15.835041
1,6045448,48173,138400,746,568,2729862,365174,1590020,774668,1975,...,506610,2015,0.133770,2.052518,1.488168,5.005144,17.062755,50.516322,15.151433,16.588745
2,1014699,47169,193500,711,613,488845,79451,275063,134331,1976,...,92836,2015,0.162528,2.047651,0.826994,4.122863,18.880574,51.377798,14.840460,17.807531
3,1869365,52997,133200,726,585,809811,73198,487948,248665,1971,...,161784,2015,0.090389,1.962271,0.712736,2.904880,17.922771,54.586397,12.363289,15.854664
4,2798636,51847,173700,973,826,1192083,175374,559793,456916,1993,...,352011,2015,0.147116,1.225155,3.634936,5.319355,14.436533,50.903226,15.233742,24.709532


In [7]:
# ---- FRED State Unemployment Rates ----
state_ur_path = os.path.join(PROCESSED_DIR, "fred_state_unemployment.parquet")

if os.path.isfile(state_ur_path):
    print(f"Loading cached: {state_ur_path}")
    df_state_ur = pd.read_parquet(state_ur_path)
else:
    from fredapi import Fred
    fred = Fred(api_key=FRED_API_KEY)
    state_ur_series = {}
    print("Fetching state-level unemployment rates from FRED...")
    for abbr in sorted(FIPS_TO_ABBR.values()):
        if abbr in ("PR", "DC"):
            continue
        sid = f"{abbr}UR"
        try:
            s = fred.get_series(sid, observation_start=f"{min(YEARS)}-01-01")
            if len(s.dropna()) > 0:
                state_ur_series[abbr] = s.dropna().iloc[-1]
        except Exception:
            pass

    df_state_ur = pd.DataFrame([
        {"state_abbr": abbr, "unemployment_rate": rate}
        for abbr, rate in state_ur_series.items()
    ])
    df_state_ur.to_parquet(state_ur_path, index=False)
    print(f"Retrieved unemployment for {len(df_state_ur)} states")

print(f"State unemployment: {len(df_state_ur)} states")
df_state_ur.sort_values("unemployment_rate", ascending=False).head(10)

Loading cached: c:\Users\hp\Desktop\Personal_Projects\pj_feb\data\processed\fred_state_unemployment.parquet
State unemployment: 50 states


,state_abbr,unemployment_rate
4,CA,5.5
30,NJ,5.4
32,NV,5.2
36,OR,5.2
7,DE,5.2
21,MI,5.0
0,AK,4.8
39,SC,4.8
18,MA,4.8
46,WA,4.7


In [8]:
# ---- Viz: State-Level Animated Choropleth (Median Income) ----
df_states_all["state_abbr"] = df_states_all["state"].astype(str).str.zfill(2).map(FIPS_TO_ABBR)
df_map_all = df_states_all.dropna(subset=["state_abbr", "median_income"]).copy()
df_map_all = df_map_all[df_map_all["median_income"] > 0]

fig_income = px.choropleth(
    df_map_all.sort_values("year"), locations="state_abbr", locationmode="USA-states",
    color="median_income", scope="usa", animation_frame="year",
    color_continuous_scale="Viridis",
    range_color=[30000, df_map_all["median_income"].quantile(0.98)],
    hover_name="NAME", hover_data={"median_income": ":$,.0f", "population": ":,.0f",
                                    "median_home_value": ":$,.0f", "state_abbr": False, "year": True},
    labels={"median_income": "Median Income ($)", "year": "Year"},
    title="Census ACS -- Median Household Income by State (2015-2024)",
)
fig_income.update_layout(geo=dict(bgcolor="rgba(0,0,0,0)"), margin=dict(l=0, r=0, t=50, b=0), height=500)
fig_income.show()

---
## 3. County Level -- ACS Demographics + BLS LAUS Unemployment

**Source**: Census ACS 5-year (28+8 vars) + BLS LAUS (county-level monthly unemployment)
**Fetch**: ACS -- 1 call per year. BLS -- chunked POST (50 series/chunk)
**Join key**: `county_fips = state + county`, `year`

In [9]:
# ---- Section 3a: County-Level ACS (2015-2024) ----
county_path = os.path.join(DATA_DIR, "census_county_demographics_2015_2024.csv")

if os.path.isfile(county_path):
    print(f"Loading cached: {county_path}")
    df_counties_all = pd.read_csv(county_path, dtype={"state": str, "county": str})
else:
    def _add_county_fips(df):
        df["county_fips"] = df["state"].astype(str).str.zfill(2) + df["county"].astype(str).str.zfill(3)
        return df

    print(f"Fetching county-level: {len(YEARS)} years, 28 vars (two-batch)")
    df_counties_all, county_errors = fetch_acs_level(
        {"for": "county:*"}, ["state", "county"],
        extra_cols_fn=_add_county_fips, label="counties"
    )
    df_counties_all.to_csv(county_path, index=False)
    print(f"Saved: {county_path}")
    if county_errors:
        print(f"Errors: {county_errors}")

df_counties_all["state"] = df_counties_all["state"].astype(str).str.zfill(2)
df_counties_all["county"] = df_counties_all["county"].astype(str).str.zfill(3)
df_counties_all["county_fips"] = df_counties_all["state"] + df_counties_all["county"]

df_counties = df_counties_all[df_counties_all["year"] == latest_year].copy()
print(f"\n--- County-Level: {len(df_counties_all):,} rows x {len(df_counties_all.columns)} cols ---")
print(f"Years: {int(df_counties_all['year'].min())}-{int(df_counties_all['year'].max())} | Counties/yr: {len(df_counties):,}")

Loading cached: c:\Users\hp\Desktop\Personal_Projects\pj_feb\data\census_county_demographics_2015_2024.csv

--- County-Level: 32,208 rows x 41 cols ---
Years: 2015-2024 | Counties/yr: 3,222


In [10]:
# ---- Section 3b: BLS LAUS County Unemployment ----
bls_county_path = os.path.join(PROCESSED_DIR, "bls_laus_county.parquet")

if os.path.isfile(bls_county_path):
    print(f"Loading cached: {bls_county_path}")
    df_bls_county = pd.read_parquet(bls_county_path)
else:
    county_fips_list = df_counties_all["county_fips"].dropna().unique().tolist()
    county_series = [f"LAUCN{fips}0000000003" for fips in county_fips_list]

    bls_url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
    bls_base = {"startyear": str(min(YEARS)), "endyear": str(max(YEARS))}
    if BLS_API_KEY:
        bls_base["registrationkey"] = BLS_API_KEY

    county_ur_records = []
    chunk_size = 50
    print(f"Fetching BLS LAUS: {len(county_series)} series in {len(county_series)//chunk_size + 1} chunks")
    for i in range(0, len(county_series), chunk_size):
        chunk = county_series[i: i + chunk_size]
        payload = {**bls_base, "seriesid": chunk}
        try:
            r_bls = requests.post(bls_url, json=payload, timeout=60)
            j = r_bls.json()
            if j.get("status") != "REQUEST_SUCCEEDED":
                msg = j.get("message", "")
                if "threshold" in str(msg).lower():
                    print(f"  Rate limit at chunk {i//chunk_size}, stopping.")
                    break
                continue
            for s in j.get("Results", {}).get("series", []):
                sid = s.get("seriesID", "")
                cfips = sid[5:10] if len(sid) >= 10 else ""
                for d in s.get("data", []):
                    county_ur_records.append({
                        "county_fips": cfips, "year": d.get("year"),
                        "period": d.get("period"),
                        "unemployment_rate": pd.to_numeric(d.get("value"), errors="coerce"),
                    })
        except Exception as e:
            print(f"  BLS chunk {i//chunk_size}: {e}")
        if (i // chunk_size) % 10 == 9:
            time.sleep(1)

    df_bls_county = pd.DataFrame(county_ur_records)
    df_bls_county = df_bls_county[
        df_bls_county["period"].astype(str).str.match(r"^M(0[1-9]|1[0-2])$")
    ].copy()
    df_bls_county.to_parquet(bls_county_path, index=False)
    print(f"Saved: {bls_county_path}")

df_bls_cty_latest = (
    df_bls_county.assign(
        year_int=pd.to_numeric(df_bls_county["year"], errors="coerce"),
        month_int=pd.to_numeric(df_bls_county["period"].str[1:], errors="coerce")
    ).sort_values(["year_int", "month_int"], ascending=[False, False])
    .drop_duplicates("county_fips", keep="first")
) if len(df_bls_county) > 0 else pd.DataFrame()

print(f"\n--- BLS County Unemployment ---")
print(f"  Records: {len(df_bls_county):,} | Counties: {df_bls_county['county_fips'].nunique() if len(df_bls_county) > 0 else 0}")

Loading cached: c:\Users\hp\Desktop\Personal_Projects\pj_feb\data\processed\bls_laus_county.parquet

--- BLS County Unemployment ---
  Records: 45,000 | Counties: 625


---
### 3c. Census County Business Patterns (CBP) -- Commercial RE Activity

**Source**: Census CBP API (NAICS 531 = Real Estate)  
**Geography**: County (same `county_fips` as ACS)  
**FMV Signal**: Commercial RE establishment count proxies local landlord competition and market depth


In [ ]:
# ---- Section 3c: Census County Business Patterns (NAICS 531) ----
cbp_path = os.path.join(PROCESSED_DIR, "cbp_naics531_county.parquet")

if os.path.isfile(cbp_path):
    print(f"Loading cached: {cbp_path}")
    df_cbp = pd.read_parquet(cbp_path)
else:
    cbp_frames = []
    for yr in YEARS:
        cbp_url = f"https://api.census.gov/data/{yr}/cbp"
        try:
            params = {
                "get": "ESTAB,EMP,PAYANN,NAICS2017",
                "for": "county:*",
                "NAICS2017": "531",
            }
            if CENSUS_API_KEY:
                params["key"] = CENSUS_API_KEY
            r = requests.get(cbp_url, params=params, timeout=60)
            r.raise_for_status()
            data = r.json()
            df_yr = pd.DataFrame(data[1:], columns=data[0])
            df_yr["year"] = yr
            cbp_frames.append(df_yr)
            print(f"  {yr}: {len(df_yr):,} county records  [OK]")
        except Exception as e:
            print(f"  {yr}: CBP error - {e}")
        time.sleep(0.3)

    if cbp_frames:
        df_cbp = pd.concat(cbp_frames, ignore_index=True)
        df_cbp["state"] = df_cbp["state"].astype(str).str.zfill(2)
        df_cbp["county"] = df_cbp["county"].astype(str).str.zfill(3)
        df_cbp["county_fips"] = df_cbp["state"] + df_cbp["county"]
        for col in ["ESTAB", "EMP", "PAYANN"]:
            df_cbp[col] = pd.to_numeric(df_cbp[col], errors="coerce")
        df_cbp.rename(columns={
            "ESTAB": "re_establishments_531",
            "EMP": "re_employment_531",
            "PAYANN": "re_annual_payroll_531",
        }, inplace=True)
        df_cbp.to_parquet(cbp_path, index=False)
        print(f"\nSaved: {cbp_path}")
    else:
        df_cbp = pd.DataFrame()
        print("CBP: no data fetched")

print(f"\n--- CBP NAICS 531 (Real Estate): {len(df_cbp):,} rows ---")
if len(df_cbp) > 0:
    print(f"  Years: {sorted(df_cbp['year'].unique())}")
    print(f"  Counties: {df_cbp['county_fips'].nunique():,}")
    for col in ['re_establishments_531', 're_employment_531', 're_annual_payroll_531']:
        if col in df_cbp.columns:
            print(f"  {col}: median={df_cbp[col].median():.0f}, mean={df_cbp[col].mean():.0f}")


In [ ]:
# ---- Viz: County-Level Income Choropleth ----
from urllib.request import urlopen
import json as _json
with urlopen("https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json") as r:
    counties_geojson = _json.load(r)

df_county_map = df_counties_all[
    (df_counties_all["median_income"] > 0) & (df_counties_all["county_fips"].notna())
].copy()

fig_county = px.choropleth(
    df_county_map.sort_values("year"), geojson=counties_geojson, locations="county_fips",
    color="median_income", scope="usa", animation_frame="year",
    color_continuous_scale="Viridis",
    range_color=[25000, df_county_map["median_income"].quantile(0.98)],
    hover_data={"median_income": ":$,.0f", "population": ":,.0f", "county_fips": True, "year": True},
    labels={"median_income": "Median Income ($)", "county_fips": "FIPS", "year": "Year"},
    title="Census ACS -- Median Household Income by County (2015-2024)",
)
fig_county.update_layout(geo=dict(bgcolor="rgba(0,0,0,0)"), margin=dict(l=0, r=0, t=50, b=0), height=550)
fig_county.show()

---
## 4. Tract Level -- ACS Demographics (2015-2024)

**Source**: Census ACS 5-year (28+8 vars) for every tract in every state
**Fetch**: 1 API call per state per year (52 states x 10 years = 520 calls, two batches each)
**Join key**: `tract_fips = state + county + tract`, `year`

In [ ]:
# ---- Section 4a: Tract-Level ACS (2015-2024) ----
tract_path = os.path.join(DATA_DIR, "census_tract_all_us_demographics_2015_2024.parquet")

if os.path.isfile(tract_path):
    print(f"Loading cached: {tract_path}")
    df_tracts_all = pd.read_parquet(tract_path)
else:
    all_tract_frames, tract_errors = [], []
    print(f"Fetching tract-level: {len(ALL_STATE_FIPS)} states x {len(YEARS)} years, 28 vars (two-batch)")
    for yr in YEARS:
        base_url = f"https://api.census.gov/data/{yr}/acs/{DATASET}"
        yr_frames = []
        for fips in ALL_STATE_FIPS:
            try:
                df_yr = fetch_census_two_batch(
                    base_url, {"for": "tract:*", "in": f"state:{fips}"},
                    CENSUS_API_KEY, RENAME_MAP, ["state", "county", "tract"],
                )
                df_yr["year"] = yr
                df_yr["tract_fips"] = df_yr["state"].astype(str).str.zfill(2) + df_yr["county"].astype(str).str.zfill(3) + df_yr["tract"]
                df_yr = compute_fmv_derived(df_yr)
                yr_frames.append(df_yr)
            except Exception as e:
                tract_errors.append(f"{yr}/{fips}: {e}")
            time.sleep(0.15)
        if yr_frames:
            df_yr_all = pd.concat(yr_frames, ignore_index=True)
            all_tract_frames.append(df_yr_all)
            print(f"  {yr}: {len(df_yr_all):,} tracts")

    df_tracts_all = pd.concat(all_tract_frames, ignore_index=True)
    df_tracts_all.to_parquet(tract_path, index=False, engine="pyarrow")
    print(f"Saved: {tract_path}")
    if tract_errors:
        print(f"Errors ({len(tract_errors)}): {tract_errors[:5]}")

if "tract_fips" not in df_tracts_all.columns:
    df_tracts_all["state"] = df_tracts_all["state"].astype(str).str.zfill(2)
    df_tracts_all["county"] = df_tracts_all["county"].astype(str).str.zfill(3)
    df_tracts_all["tract_fips"] = df_tracts_all["state"] + df_tracts_all["county"] + df_tracts_all["tract"]

df_tracts_latest = df_tracts_all[df_tracts_all["year"] == int(df_tracts_all["year"].max())].copy()
print(f"\n--- Tract-Level: {len(df_tracts_all):,} rows x {len(df_tracts_all.columns)} cols ---")
print(f"Years: {int(df_tracts_all['year'].min())}-{int(df_tracts_all['year'].max())} | Tracts/yr: ~{len(df_tracts_latest):,}")

---
### 4b. TIGER/Line Tract Shapefiles + Spatial Join

**Source**: `https://www2.census.gov/geo/tiger/TIGER{year}/TRACT/`
Downloads TIGER/Line tract shapefiles and builds a `geocode_bulk()` function for offline spatial joins.
Uses a single TIGER vintage (2024) -- county/tract boundaries are stable across 2015-2024.

In [ ]:
# ---- Section 4b: TIGER/Line Tract Shapefiles + Spatial Join ----
tiger_parquet = os.path.join(PROCESSED_DIR, "tiger_tracts.parquet")

if os.path.isfile(tiger_parquet):
    print(f"Loading cached: {tiger_parquet}")
    tracts_gdf = gpd.read_parquet(tiger_parquet)
else:
    tiger_year = "2024"
    tiger_base_url = f"https://www2.census.gov/geo/tiger/TIGER{tiger_year}/TRACT"
    tiger_dir = os.path.join(RAW_DIR, "tiger")
    os.makedirs(tiger_dir, exist_ok=True)
    all_tract_gdfs, tiger_errors = [], []
    print(f"Downloading TIGER/Line {tiger_year} tract shapefiles for {len(ALL_STATE_FIPS)} states...")
    for idx, st in enumerate(ALL_STATE_FIPS):
        fname = f"tl_{tiger_year}_{st}_tract.zip"
        local_zip = os.path.join(tiger_dir, fname)
        url = f"{tiger_base_url}/{fname}"
        if not os.path.isfile(local_zip):
            try:
                r = requests.get(url, stream=True, timeout=120)
                r.raise_for_status()
                with open(local_zip, "wb") as f:
                    for chunk in r.iter_content(chunk_size=65536):
                        f.write(chunk)
            except Exception as e:
                tiger_errors.append(f"State {st}: {e}")
                continue
        try:
            gdf = gpd.read_file(f"zip://{local_zip}")
            keep_cols = [c for c in ["STATEFP","COUNTYFP","TRACTCE","GEOID","NAMELSAD",
                                      "ALAND","AWATER","INTPTLAT","INTPTLON","geometry"] if c in gdf.columns]
            all_tract_gdfs.append(gdf[keep_cols])
        except Exception as e:
            tiger_errors.append(f"State {st}: read error - {e}")
        if (idx + 1) % 10 == 0:
            print(f"  Loaded {idx + 1}/{len(ALL_STATE_FIPS)} states...")

    tracts_gdf = gpd.GeoDataFrame(pd.concat(all_tract_gdfs, ignore_index=True)).to_crs("EPSG:4326")
    tracts_gdf.to_parquet(tiger_parquet, index=False)
    print(f"Saved: {tiger_parquet} ({os.path.getsize(tiger_parquet)/1024/1024:.0f} MB)")
    if tiger_errors:
        print(f"Errors: {tiger_errors[:5]}")

print(f"\n--- TIGER/Line: {len(tracts_gdf):,} tracts, {tracts_gdf['STATEFP'].nunique()} states ---")

def geocode_bulk(df_input, lat_col="lat", lon_col="lon", tracts=tracts_gdf):
    """Spatial join: assign FIPS tract to every row with lat/lon coordinates."""
    gdf = gpd.GeoDataFrame(df_input, geometry=gpd.points_from_xy(df_input[lon_col], df_input[lat_col]), crs="EPSG:4326")
    result = gpd.sjoin(gdf, tracts[["STATEFP","COUNTYFP","TRACTCE","GEOID","geometry"]], how="left", predicate="within")
    result = result.rename(columns={"STATEFP": "state_fips", "COUNTYFP": "county_fips",
                                     "TRACTCE": "tract", "GEOID": "geoid"})
    return result.drop(columns=["geometry", "index_right"], errors="ignore")

print("geocode_bulk() available for coordinate-to-tract lookups")

# Compute population_density for tracts using TIGER land area
if len(df_tracts_all) > 0 and len(tracts_gdf) > 0:
    land_area = tracts_gdf[["GEOID", "ALAND"]].copy()
    land_area["ALAND"] = pd.to_numeric(land_area["ALAND"], errors="coerce")
    land_area.rename(columns={"GEOID": "tract_fips"}, inplace=True)
    if "ALAND" not in df_tracts_all.columns:
        df_tracts_all = df_tracts_all.merge(land_area, on="tract_fips", how="left")
    aland_sqkm = (df_tracts_all["ALAND"] / 1e6).replace(0, np.nan)
    df_tracts_all["population_density"] = df_tracts_all["population"] / aland_sqkm
    print(f"  population_density: {df_tracts_all['population_density'].notna().sum():,} tracts enriched")

---
### 4c. FHFA House Price Index (ZIP + Tract Level)

**Source**: FHFA DataTools (CSV/XLSX)
Downloads ZIP-level and tract-level House Price Index data for real estate appreciation trends.

In [ ]:
# ---- Section 4c: FHFA HPI Download ----
from io import StringIO, BytesIO

zip_hpi_parquet = os.path.join(PROCESSED_DIR, "fhfa_hpi_zip.parquet")
tract_hpi_parquet = os.path.join(PROCESSED_DIR, "fhfa_hpi_tract.parquet")

fhfa_candidates = [
    ("zip", "csv", "https://www.fhfa.gov/DataTools/Downloads/Documents/HPI/HPI_AT_BDL_ZIP5.csv"),
    ("zip", "xlsx", "https://www.fhfa.gov/DataTools/Downloads/Documents/HPI/HPI_AT_BDL_ZIP5.xlsx"),
    ("tract", "csv", "https://www.fhfa.gov/DataTools/Downloads/Documents/HPI/HPI_AT_BDL_tract.csv"),
    ("master", "csv", "https://www.fhfa.gov/HPI_master.csv"),
    ("master", "csv", "https://www.fhfa.gov/DataTools/Downloads/Documents/HPI/HPI_AT_BDL.csv"),
]
fhfa_log = []
got_zip = os.path.isfile(zip_hpi_parquet)
got_tract = os.path.isfile(tract_hpi_parquet)

for level, fmt, url in fhfa_candidates:
    if got_zip and got_tract:
        break
    try:
        r = requests.get(url, timeout=120, headers={"User-Agent": "Mozilla/5.0"})
        r.raise_for_status()
        df_hpi = pd.read_csv(StringIO(r.text)) if fmt == "csv" else pd.read_excel(BytesIO(r.content))
        fhfa_log.append(f"[OK] {url} -> {len(df_hpi):,} rows")

        if level == "master" and "level" in df_hpi.columns:
            if not got_zip:
                z = df_hpi[df_hpi["level"].str.contains("ZIP", case=False, na=False)]
                if len(z) > 0:
                    z.to_parquet(zip_hpi_parquet, index=False); got_zip = True
                    fhfa_log.append(f"  -> ZIP extracted: {len(z):,}")
            if not got_tract:
                t = df_hpi[df_hpi["level"].str.contains("tract|Tract", case=False, na=False)]
                if len(t) > 0:
                    t.to_parquet(tract_hpi_parquet, index=False); got_tract = True
                    fhfa_log.append(f"  -> Tract extracted: {len(t):,}")
        elif level == "zip" and not got_zip:
            df_hpi.to_parquet(zip_hpi_parquet, index=False); got_zip = True
        elif level == "tract" and not got_tract:
            df_hpi.to_parquet(tract_hpi_parquet, index=False); got_tract = True
    except Exception as e:
        fhfa_log.append(f"[FAIL] {url} -> {e}")

print("--- FHFA HPI ---")
for item in fhfa_log:
    print(f"  {item}")
for label, p in [("ZIP", zip_hpi_parquet), ("Tract", tract_hpi_parquet)]:
    if os.path.isfile(p):
        df_check = pd.read_parquet(p)
        print(f"  {label}: {len(df_check):,} rows x {len(df_check.columns)} cols")
    else:
        print(f"  {label}: not available")

---
### 4d. OpenCelliD Cell Tower Data (Bulk, Nationwide)

**Source**: OpenCelliD bulk download (~500 MB compressed)
Filters to US towers (MCC 310-316), aggregates to tract level for tower density features.

In [ ]:
# ---- Section 4d: OpenCelliD Cell Towers + Tract Aggregation ----
US_MCC_CODES = [310, 311, 312, 313, 316]
oci_processed_path = os.path.join(PROCESSED_DIR, "opencellid_us.parquet")
oci_agg_path = os.path.join(PROCESSED_DIR, "opencellid_tract_agg.parquet")

if os.path.isfile(oci_processed_path):
    print(f"Loading cached: {oci_processed_path}")
    df_towers = pd.read_parquet(oci_processed_path)
else:
    raw_dir = os.path.join(RAW_DIR, "opencellid")
    os.makedirs(raw_dir, exist_ok=True)
    gz_path = os.path.join(raw_dir, "cell_towers.csv.gz")

    if not OPENCELLID_API_KEY:
        print("Set OPENCELLID_API_KEY in .env"); df_towers = pd.DataFrame()
    else:
        if not os.path.isfile(gz_path):
            url = f"https://opencellid.org/ocid/downloads?token={OPENCELLID_API_KEY}&type=full&file=cell_towers.csv.gz"
            print("Downloading OpenCelliD bulk database (~500 MB)...")
            r_oci = requests.get(url, stream=True, timeout=600)
            r_oci.raise_for_status()
            with open(gz_path, "wb") as f:
                for chunk in r_oci.iter_content(chunk_size=262144):
                    f.write(chunk)
            print(f"Downloaded: {os.path.getsize(gz_path)/1024/1024:.0f} MB")

        print(f"Filtering to US towers (MCC in {US_MCC_CODES})...")
        usecols = ["radio","mcc","net","area","cell","lon","lat","range","samples","changeable","created","updated"]
        us_frames = []
        for chunk in pd.read_csv(gz_path, compression="gzip", usecols=usecols,
                                  dtype={"mcc":"Int64","net":"Int64","area":"Int64","cell":"Int64"}, chunksize=500_000):
            us_chunk = chunk[chunk["mcc"].isin(US_MCC_CODES)]
            if len(us_chunk) > 0:
                us_frames.append(us_chunk)
        df_towers = pd.concat(us_frames, ignore_index=True) if us_frames else pd.DataFrame()
        if len(df_towers) > 0:
            df_towers.to_parquet(oci_processed_path, index=False)
            print(f"Saved {len(df_towers):,} US towers -> {oci_processed_path}")

if "lng" in df_towers.columns and "lon" not in df_towers.columns:
    df_towers.rename(columns={"lng": "lon"}, inplace=True)

if len(df_towers) > 0:
    print(f"\n--- OpenCelliD: {len(df_towers):,} US towers ---")
    print(df_towers["radio"].value_counts().to_string())

# Tract-level aggregation via spatial join
if os.path.isfile(oci_agg_path):
    print(f"Loading cached: {oci_agg_path}")
    oci_agg = pd.read_parquet(oci_agg_path)
elif len(df_towers) > 0 and len(tracts_gdf) > 0:
    vz_nets = {311, 312}; att_nets = {310}; tmo_nets = {316}
    gdf_towers = gpd.GeoDataFrame(df_towers, geometry=gpd.points_from_xy(df_towers["lon"], df_towers["lat"]), crs="EPSG:4326")
    joined = gpd.sjoin(gdf_towers, tracts_gdf[["GEOID","ALAND","geometry"]], how="inner", predicate="within")
    oci_agg = joined.groupby("GEOID").agg(
        tower_count=("cell", "count"),
        lte_count=("radio", lambda x: (x == "LTE").sum()),
        nr_count=("radio", lambda x: (x == "NR").sum()),
        has_verizon=("net", lambda x: bool(set(x.dropna().astype(int)) & vz_nets)),
        has_att=("net", lambda x: bool(set(x.dropna().astype(int)) & att_nets)),
        has_tmobile=("net", lambda x: bool(set(x.dropna().astype(int)) & tmo_nets)),
        unique_carriers=("net", "nunique"),
    ).reset_index()
    oci_agg.rename(columns={"GEOID": "tract_fips"}, inplace=True)
    land_area = tracts_gdf[["GEOID","ALAND"]].rename(columns={"GEOID": "tract_fips"})
    land_area["ALAND"] = pd.to_numeric(land_area["ALAND"], errors="coerce")
    oci_agg = oci_agg.merge(land_area, on="tract_fips", how="left")
    oci_agg["tower_density_sqkm"] = oci_agg["tower_count"] / (oci_agg["ALAND"] / 1e6).replace(0, np.nan)
    oci_agg["pct_5g_nr"] = oci_agg["nr_count"] / oci_agg["tower_count"].replace(0, np.nan)
    oci_agg.to_parquet(oci_agg_path, index=False)
    print(f"Saved: {oci_agg_path} ({len(oci_agg):,} tracts with towers)")
else:
    oci_agg = pd.DataFrame()
    print("Skipping tower aggregation (no towers or tracts)")

---
## 5. Block Group Level -- ACS Demographics (Latest Year)

**Source**: Census ACS 5-year (28+8 vars) at block group level (~220k block groups)
**Fetch**: 1 API call per state per year (52 calls). Finest ACS geography.
Set `BG_YEARS` to extend to the full panel (very large).

In [ ]:
# ---- Section 5: Block Group Level ACS ----
BG_YEARS = [latest_year]

all_bg_frames = []
for bg_year in BG_YEARS:
    bg_path = os.path.join(DATA_DIR, f"census_blockgroup_all_us_{bg_year}.parquet")
    if os.path.isfile(bg_path):
        print(f"Loading cached: {bg_path}")
        df_bg = pd.read_parquet(bg_path)
    else:
        bg_frames, bg_errors = [], []
        base_url = f"https://api.census.gov/data/{bg_year}/acs/{DATASET}"
        print(f"Fetching block groups: {len(ALL_STATE_FIPS)} states, year {bg_year}, 28 vars")
        for idx, st in enumerate(ALL_STATE_FIPS):
            try:
                df_m = fetch_census_two_batch(
                    base_url,
                    {"for": "block group:*", "in": f"state:{st} county:*"},
                    CENSUS_API_KEY, RENAME_MAP,
                    ["state", "county", "tract", "block group"],
                )
                bg_frames.append(df_m)
            except Exception as e:
                bg_errors.append(f"State {st}: {e}")
            if (idx + 1) % 10 == 0:
                print(f"  {idx + 1}/{len(ALL_STATE_FIPS)} states ({bg_year})...")

        if bg_frames:
            df_bg = pd.concat(bg_frames, ignore_index=True)
            df_bg["state"] = df_bg["state"].astype(str).str.zfill(2)
            df_bg["county"] = df_bg["county"].astype(str).str.zfill(3)
            df_bg["tract_fips"] = df_bg["state"] + df_bg["county"] + df_bg["tract"]
            df_bg["county_fips"] = df_bg["state"] + df_bg["county"]
            df_bg["year"] = bg_year
            df_bg = compute_fmv_derived(df_bg)
            df_bg.to_parquet(bg_path, index=False, engine="pyarrow")
            print(f"Saved: {bg_path}")
            if bg_errors:
                print(f"Errors: {bg_errors[:3]}")
        else:
            df_bg = pd.DataFrame()
            print(f"Block group fetch failed for {bg_year}")

    if len(df_bg) > 0:
        all_bg_frames.append(df_bg)

df_blockgroups = pd.concat(all_bg_frames, ignore_index=True) if all_bg_frames else pd.DataFrame()
print(f"\n--- Block Group ({', '.join(str(y) for y in BG_YEARS)}): {len(df_blockgroups):,} rows x {len(df_blockgroups.columns)} cols ---")

---
## 6. ZCTA (ZIP Code) Level -- ACS Demographics (2015-2024)

**Source**: Census ACS 5-year (28+8 vars) at ZCTA level (~33k ZCTAs)
**Fetch**: 1 API call per year (all ZCTAs in one call, no state filter needed)
**Join key**: `zcta`, `year`

In [ ]:
# ---- Section 6: ZCTA/ZIP-Level ACS (2015-2024) ----
zcta_path = os.path.join(DATA_DIR, "census_zcta_all_us_2015_2024.parquet")

if os.path.isfile(zcta_path):
    print(f"Loading cached: {zcta_path}")
    df_zcta_all = pd.read_parquet(zcta_path)
else:
    all_zcta_frames, zcta_errors = [], []
    print(f"Fetching ZCTA-level: {len(YEARS)} years, 28 vars (two-batch)")
    for yr in YEARS:
        base_url = f"https://api.census.gov/data/{yr}/acs/{DATASET}"
        try:
            df_yr = fetch_census_two_batch(
                base_url, {"for": "zip code tabulation area:*"},
                CENSUS_API_KEY, RENAME_MAP, ["zip code tabulation area"],
            )
            zcta_col = [c for c in df_yr.columns if "zip code" in c.lower() or "zcta" in c.lower()]
            if zcta_col:
                df_yr.rename(columns={zcta_col[0]: "zcta"}, inplace=True)
            df_yr["year"] = yr
            df_yr = compute_fmv_derived(df_yr)
            all_zcta_frames.append(df_yr)
            print(f"  {yr}: {len(df_yr):,} ZCTAs")
        except Exception as e:
            zcta_errors.append(f"{yr}: {e}")
            print(f"  {yr}: ERROR - {e}")
        time.sleep(0.5)

    df_zcta_all = pd.concat(all_zcta_frames, ignore_index=True) if all_zcta_frames else pd.DataFrame()
    if len(df_zcta_all) > 0:
        df_zcta_all.to_parquet(zcta_path, index=False, engine="pyarrow")
        print(f"Saved: {zcta_path}")
    if zcta_errors:
        print(f"Errors: {zcta_errors}")

df_zcta = df_zcta_all[df_zcta_all["year"] == latest_year].copy() if len(df_zcta_all) > 0 else pd.DataFrame()
print(f"\n--- ZCTA-Level: {len(df_zcta_all):,} rows x {len(df_zcta_all.columns)} cols ---")
if len(df_zcta_all) > 0:
    print(f"Years: {int(df_zcta_all['year'].min())}-{int(df_zcta_all['year'].max())} | ZCTAs/yr: ~{len(df_zcta):,}")

---
## 7. Temporal Trend Features -- Growth Rates (1yr / 3yr / 5yr)

Computes growth rates from the 10-year Census ACS tract panel for FMV-relevant metrics.
Trends capture market dynamics: a tract with 30% income growth over 5 years is fundamentally
different from one that stagnated, even if current income is identical.

In [ ]:
# ---- Section 7: Temporal Trend Features (Tract-Level) ----
if len(df_tracts_all) > 0:
    lt_year = int(df_tracts_all["year"].max())
    trend_metrics = {
        "population": "population", "median_income": "median_income",
        "median_home_value": "median_home_value", "median_rent": "median_rent",
        "median_contract_rent": "median_contract_rent",
        "total_housing_units": "total_housing_units",
        "vacant_housing_units": "vacant_housing_units",
        "owner_occupied_units": "owner_occupied_units",
        "renter_occupied_units": "renter_occupied_units",
        "median_monthly_housing_cost": "median_monthly_housing_cost",
        "poverty_population": "poverty_population",
        "in_labor_force": "in_labor_force",
        "vacancy_rate": "vacancy_rate", "college_degree_pct": "college_degree_pct",
        "labor_force_participation": "labor_force_participation",
        "poverty_rate": "poverty_rate",
    }
    available = {k: v for k, v in trend_metrics.items() if v in df_tracts_all.columns}
    print(f"Computing trends for {len(available)} metrics | Latest: {lt_year}")

    windows = {"1yr": 1, "3yr": 3, "5yr": 5}
    trend_series = []
    for wname, yrs_back in windows.items():
        cmp_year = lt_year - yrs_back
        df_lt = df_tracts_all[df_tracts_all["year"] == lt_year].set_index("tract_fips")
        df_cmp = df_tracts_all[df_tracts_all["year"] == cmp_year].set_index("tract_fips")
        if len(df_cmp) == 0:
            print(f"  No data for {cmp_year}, skipping {wname}"); continue
        for mname, col in available.items():
            if col not in df_lt.columns or col not in df_cmp.columns:
                continue
            lt_vals = pd.to_numeric(df_lt[col], errors="coerce")
            cmp_vals = pd.to_numeric(df_cmp[col], errors="coerce").reindex(lt_vals.index)
            denom = cmp_vals.where(cmp_vals.abs() > 1e-6, np.nan)
            growth = ((lt_vals - cmp_vals) / denom).clip(-0.9, 5.0)
            trend_series.append(growth.rename(f"{mname}_growth_{wname}"))

    if trend_series:
        df_trends = pd.concat(trend_series, axis=1).reset_index()
        df_trends.rename(columns={"index": "tract_fips"}, inplace=True)
        trends_path = os.path.join(PROCESSED_DIR, "temporal_trends_tract.parquet")
        df_trends.to_parquet(trends_path, index=False)
        num_cols = [c for c in df_trends.columns if c != "tract_fips"]
        print(f"\n--- Temporal Trends: {len(df_trends):,} tracts x {len(num_cols)} features ---")
        for c in num_cols[:6]:
            v = df_trends[c].dropna()
            print(f"  {c:45s} median={v.median():+.3f} mean={v.mean():+.3f}")
        if len(num_cols) > 6:
            print(f"  ... +{len(num_cols)-6} more")
        print(f"Saved: {trends_path}")
    else:
        df_trends = pd.DataFrame()
        print("No trends computed")
else:
    df_trends = pd.DataFrame()
    print("Skipping trends (no tract data)")

---
## 8. Hierarchical Missing Value Imputation

Fills Census-suppressed values using the geographic hierarchy:
- **States** <- national median by year
- **Counties** <- state median by year
- **Tracts** <- county median <- state median
- **ZCTAs** <- national median by year
- **BLS counties** <- FRED state-level unemployment rate

Recomputes derived features after imputation.

In [ ]:
# ---- Section 8: Hierarchical Imputation ----
RAW_NUMERIC_COLS = [c for c in RENAME_MAP.values() if c != "NAME"]

def impute_hierarchical(df, parent_agg, merge_keys, cols):
    """Fill NaN values in df from parent_agg using merge_keys."""
    for c in [x for x in cols if x in df.columns and x in parent_agg.columns]:
        pc = f"_parent_{c}"
        parent_c = parent_agg[merge_keys + [c]].rename(columns={c: pc})
        merged = df.merge(parent_c, on=merge_keys, how="left")
        m = df[c].isna() & merged[pc].notna()
        if m.any():
            df.loc[m, c] = merged.loc[m, pc].values
    return df

# 1) States: fill from national median by year
if len(df_states_all) > 0:
    for yr in df_states_all["year"].unique():
        mask = df_states_all["year"] == yr
        for c in [x for x in RAW_NUMERIC_COLS if x in df_states_all.columns]:
            national_median = df_states_all.loc[mask, c].median()
            if pd.notna(national_median):
                fill_mask = mask & df_states_all[c].isna()
                if fill_mask.any():
                    df_states_all.loc[fill_mask, c] = national_median
    df_states_all = compute_fmv_derived(df_states_all)
    print(f"[OK] States: imputed + derived | {len(df_states_all):,} rows")

# 2) Counties: fill from state median by year
if len(df_counties_all) > 0 and len(df_states_all) > 0:
    state_agg = df_states_all.groupby(["state", "year"])[RAW_NUMERIC_COLS].median().reset_index()
    for c in [x for x in RAW_NUMERIC_COLS if x in df_counties_all.columns and x in state_agg.columns]:
        pc = f"_p_{c}"
        state_agg_c = state_agg[["state", "year", c]].rename(columns={c: pc})
        merged = df_counties_all.merge(state_agg_c, on=["state", "year"], how="left")
        m = df_counties_all[c].isna() & merged[pc].notna()
        if m.any():
            df_counties_all.loc[m, c] = merged.loc[m, pc].values
    df_counties_all = compute_fmv_derived(df_counties_all)
    print(f"[OK] Counties: imputed from state + derived | {len(df_counties_all):,} rows")

# 3) Tracts: fill from county, then state
if len(df_tracts_all) > 0 and len(df_counties_all) > 0:
    if "county_fips" not in df_tracts_all.columns:
        df_tracts_all["county_fips"] = df_tracts_all["state"].astype(str).str.zfill(2) + df_tracts_all["county"].astype(str).str.zfill(3)
    county_agg = df_counties_all.groupby(["county_fips", "year"])[RAW_NUMERIC_COLS].median().reset_index()
    df_tracts_all = impute_hierarchical(df_tracts_all, county_agg, ["county_fips", "year"], RAW_NUMERIC_COLS)
    if len(df_states_all) > 0:
        state_agg2 = df_states_all.groupby(["state", "year"])[RAW_NUMERIC_COLS].median().reset_index()
        df_tracts_all = impute_hierarchical(df_tracts_all, state_agg2, ["state", "year"], RAW_NUMERIC_COLS)
    df_tracts_all = compute_fmv_derived(df_tracts_all)
    print(f"[OK] Tracts: imputed from county+state + derived | {len(df_tracts_all):,} rows")

# 4) ZCTAs: fill from national median
if len(df_zcta_all) > 0:
    for yr in df_zcta_all["year"].unique():
        mask = df_zcta_all["year"] == yr
        for c in [x for x in RAW_NUMERIC_COLS if x in df_zcta_all.columns]:
            nat_med = df_zcta_all.loc[mask, c].median()
            if pd.notna(nat_med):
                fill_mask = mask & df_zcta_all[c].isna()
                if fill_mask.any():
                    df_zcta_all.loc[fill_mask, c] = nat_med
    df_zcta_all = compute_fmv_derived(df_zcta_all)
    print(f"[OK] ZCTA: national median fallback + derived | {len(df_zcta_all):,} rows")

# 5) BLS county: fill missing unemployment from FRED state rate (across ALL years)
if len(df_bls_county) > 0 and len(df_state_ur) > 0:
    try:
        abbr_to_fips = {v: k for k, v in FIPS_TO_ABBR.items()}
        fred_ur = df_state_ur.copy()
        fred_ur["state_fips"] = fred_ur["state_abbr"].map(abbr_to_fips)
        if "state_fips" not in df_bls_county.columns:
            df_bls_county["state_fips"] = df_bls_county["county_fips"].str[:2]
        missing = df_bls_county[df_bls_county["unemployment_rate"].isna()]
        if len(missing) > 0:
            merged = missing.merge(fred_ur[["state_fips", "unemployment_rate"]].rename(
                columns={"unemployment_rate": "ur_fred"}), on="state_fips", how="left")
            fill_mask = merged["ur_fred"].notna()
            if fill_mask.any():
                idx = missing.index[fill_mask]
                df_bls_county.loc[idx, "unemployment_rate"] = merged.loc[fill_mask, "ur_fred"].values
                print(f"[OK] BLS county: {fill_mask.sum()} filled from FRED state UR")
    except Exception as e:
        print(f"[WARN] BLS imputation: {e}")

# Refresh latest-year slices after imputation
df_states = df_states_all[df_states_all["year"] == latest_year].copy()
if len(df_counties_all) > 0:
    df_counties = df_counties_all[df_counties_all["year"] == latest_year].copy()
if len(df_tracts_all) > 0:
    df_tracts_latest = df_tracts_all[df_tracts_all["year"] == int(df_tracts_all["year"].max())].copy()
if len(df_zcta_all) > 0:
    df_zcta = df_zcta_all[df_zcta_all["year"] == latest_year].copy()

df_bls_cty_latest = (
    df_bls_county.assign(
        year_int=pd.to_numeric(df_bls_county["year"], errors="coerce"),
        month_int=pd.to_numeric(df_bls_county["period"].str[1:], errors="coerce")
    ).sort_values(["year_int", "month_int"], ascending=[False, False])
    .drop_duplicates("county_fips", keep="first")
) if len(df_bls_county) > 0 else pd.DataFrame()

print("\n[OK] All hierarchical imputation complete. Latest-year slices refreshed.")

---
## 9. Summary -- Pipeline Inventory & Data Quality

In [ ]:
# ---- Section 9: Pipeline Summary ----
print("=" * 80)
print("  FULL DATA PIPELINE SUMMARY  (28 Raw + 8 Derived ACS features)")
print("=" * 80)

_safe = lambda name: globals().get(name, pd.DataFrame())
n_yr = lambda df: df["year"].nunique() if "year" in df.columns and len(df) > 0 else 0

_st   = _safe("df_states_all")
_cty  = _safe("df_counties_all")
_tr   = _safe("df_tracts_all")
_z    = _safe("df_zcta_all")
_bg   = _safe("df_blockgroups")
_bls  = _safe("df_bls_county")
_trnd = _safe("df_trends")
_oci  = _safe("oci_agg")
_tgr  = globals().get("tracts_gdf", pd.DataFrame())
_tw   = _safe("df_towers")

hpi_ok = os.path.isfile(os.path.join(PROCESSED_DIR, "fhfa_hpi_zip.parquet"))

print(f"\n  {'Source':<25} | {'Level':<15} | {'Records':>12} | {'Time':<10} | Features")
print(f"  {'-'*25}+{'-'*17}+{'-'*14}+{'-'*12}+{'-'*9}")
print(f"  {'Census ACS':<25} | {'State':<15} | {len(_st):>12,} | {n_yr(_st)} years    | 28+8")
print(f"  {'Census ACS':<25} | {'County':<15} | {len(_cty):>12,} | {n_yr(_cty)} years    | 28+8")
print(f"  {'Census ACS':<25} | {'Tract (ALL US)':<15} | {len(_tr):>12,} | {n_yr(_tr)} years    | 28+8")
print(f"  {'Census ACS':<25} | {'ZCTA (ZIP)':<15} | {len(_z):>12,} | {n_yr(_z)} years    | 28+8")
print(f"  {'Census ACS':<25} | {'Block Group':<15} | {len(_bg):>12,} | latest yr  | 28+8")
print(f"  {'TIGER/Line':<25} | {'Tract polygons':<15} | {len(_tgr):>12,} | -          | geom")
print(f"  {'BLS LAUS':<25} | {'County unemp.':<15} | {len(_bls):>12,} | monthly    | 1")
print(f"  {'FHFA HPI':<25} | {'ZIP + Tract':<15} | {'available' if hpi_ok else 'pending':>12} | quarterly  | HPI")
print(f"  {'OpenCelliD (bulk)':<25} | {'Lat/Lon':<15} | {len(_tw):>12,} | current    | towers")
print(f"  {'OpenCelliD (tract agg)':<25} | {'Tract':<15} | {len(_oci):>12,} | current    | density")
n_trend_cols = len([c for c in _trnd.columns if c != "tract_fips"]) if len(_trnd) > 0 else 0
print(f"  {'Temporal Trends':<25} | {'Tract':<15} | {len(_trnd):>12,} | 1/3/5yr    | {n_trend_cols}")
print(f"\n  Timeline: {min(YEARS)}-{max(YEARS)} ({len(YEARS)} ACS vintages)")
print("=" * 80)